In [1]:
# импортируем библиотеку numpy
import numpy as np

In [2]:
# пишем функцию, вычисляющую среднее расстояние
# между двумя значениями
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [3]:
# создаем массивы
x = np.random.randn(10000000)
y = np.random.randn(10000000)

In [4]:
# проверяем скорость выполнения нашей функции
%timeit mean_distance(x, y)

2.5 s ± 31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# проверяем скорость выполнения функции np.mean()
%timeit np.mean(x - y)

22.9 ms ± 597 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# импортируем numba
import numba as nb

In [7]:
# превращаем нашу функцию в компилируемую функцию
# Numba с помощью функции numba.jit()
numba_mean_distance = nb.jit(mean_distance)

In [8]:
# проверяем скорость выполнения 
# функции numba_mean_distance()
%timeit numba_mean_distance(x, y)

12.6 ms ± 701 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# воспользуемся декоратором @jit
@nb.jit

def nb_mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [10]:
# проверяем скорость выполнения функции nb_mean_distance()
%timeit nb_mean_distance(x, y)

11.9 ms ± 433 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
# воспользуемся декоратором @vectorize
@nb.vectorize()

def nb_square(x):
    return x ** 2

In [12]:
# проверяем скорость выполнения функции nb_square()
%timeit nb_square(x)

14.8 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# генерируем ряд значений
data = np.random.randn(10000000)

In [14]:
# воспользуемся декоратором @jit без распараллеливания
@nb.jit()

def f(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

%timeit f(data)

124 ms ± 5.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# воспользуемся декоратором @jit с распараллеливанием
@nb.jit(parallel=True)

def f(x):
    return np.cos(x) ** 2 + np.sin(x) ** 2

%timeit f(data)

122 ms ± 4.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# воспользуемся декоратором @jit без 
# распараллеливания и функцией prange()
@nb.jit()

def compute(x):
    s = 0
    for i in nb.prange(x.shape[0]):
        s += x[i]
    return s

%timeit compute(data)

11.2 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
# воспользуемся декоратором @jit c 
# распараллеливанием и функцией prange()
@nb.jit(parallel=True)

def compute(x):
    s = 0
    for i in nb.prange(x.shape[0]):
        s += x[i]
    return s

%timeit compute(data)

11.7 ms ± 92 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np

# создаем серию с 100000 значениями
series = pd.Series(np.random.randn(100000))

# задаем объект Rolling
roll = series.rolling(10)
# пишем функцию вычисления среднего
def f(x):
    return np.mean(x)

In [19]:
# запускаем в первый раз, время компиляции 
# повлияет на производительность, здесь и
# далее если raw=True, то переданная в метод .apply() 
# функция вместо серии получает массив NumPy,
# -r(epeat) - сколько раз повторять таймер,
# -n(umber) - сколько раз выполнять команду
%timeit -r 1 -n 1 roll.apply(f, engine='numba', raw=True)

434 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
# функция кэшируется и производительность улучшается
%timeit roll.apply(f, engine='numba', raw=True)

14.5 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit roll.apply(f, raw=True)

690 ms ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
